In [5]:
import pandas as pd
import os

In [ ]:
def extract_sector_name_from_filename(file_path):
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    if base_name.endswith(" Multiples"):
        sector_name = base_name.replace(" Multiples", "")
    else:
        sector_name = base_name
    return sector_name

def transform_to_panel(file_path):
    sector_name = extract_sector_name_from_filename(file_path)
    df = pd.read_csv(file_path, sep=';')
    year_columns = [col for col in df.columns if col.isdigit()]
    melted_df = pd.melt(
        df,
        id_vars=['Ticker', 'Field Mnemonic', 'Data State'],
        value_vars=year_columns,
        var_name='Year',
        value_name='Value'
    )
    melted_df['Sector'] = sector_name
    melted_df['Year'] = melted_df['Year'].astype(int)
    melted_df = melted_df.dropna(subset=['Value'])
    
    pivot_df = melted_df.pivot_table(
        index=['Ticker', 'Data State', 'Year', 'Sector'],
        columns='Field Mnemonic',
        values='Value',
        aggfunc='first'
    ).reset_index()
    
    required_columns = ['BEST_CUR_EV_TO_EBITDA', 'EV_TO_T12M_EBITDA', 'EV_TO_T12M_SALES']
    for col in required_columns:
        if col not in pivot_df.columns:
            pivot_df[col] = None  # Add missing columns with NaN values

    final_columns = ['Ticker', 'Data State', 'Year', 'Sector', 
                     'BEST_CUR_EV_TO_EBITDA', 'EV_TO_T12M_EBITDA', 'EV_TO_T12M_SALES']
    pivot_df = pivot_df[final_columns]
    
    return pivot_df

def process_files_in_directory(input_dir, output_dir):
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_dir, file_name)
            print(f"Processing file: {file_name}")
            try:
                panel_data = transform_to_panel(file_path)
                output_file_name = f"{os.path.splitext(file_name)[0]} Panel.csv"
                output_path = os.path.join(output_dir, output_file_name)
                panel_data.to_csv(output_path, index=False)
                print(f"File saved: {output_path}")
            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

input_directory = r"C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\python codes\multiples\to be cleaned"
output_directory = os.path.join(input_directory, "output")

os.makedirs(output_directory, exist_ok=True)

process_files_in_directory(input_directory, output_directory)


Processing file: Hardware. Data Networking Equipment Dashboard.csv
File saved: C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\python codes\multiples\to be cleaned\output\Hardware. Data Networking Equipment Dashboard Panel.csv
Processing file: Hardware. Global Display Components.csv
File saved: C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\python codes\multiples\to be cleaned\output\Hardware. Global Display Components Panel.csv
Processing file: Hardware. Global Display Panels Compeitive peers.csv
File saved: C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\python codes\multiples\to be cleaned\output\Hardware. Global Display Panels Compeitive peers Panel.csv
Processing file: Hardware. Global Displays (TV Manufacturers) Competitive Peers.csv
File saved: C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\python codes\multiples\to be cleaned\output\Hardware. Global Displays (TV Manufacturers) Competitive Peers Panel.csv
Processing file: Hardwar